# Bibliotecas e variaveis de iniciação

In [14]:
import datetime
import glob
import os

import pandas as pd
import numpy as np

from google.colab import drive

import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
drive.mount('/content/drive')

dir_arq_resultado_c = "/content/drive/My Drive/datathon/resultados_csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Arquivo | itens-parte.csv

## Parte 1 - Geração dataframe | df_itens_juncao

In [ ]:
df_itens_juncao = pd.read_csv("/content/drive/My Drive/datathon/resultados_csv/itens_final.csv")

In [ ]:
abreviacoes_estados = {
    'sao-paulo': 'sp',
    'rio-de-janeiro': 'rj',
    'minas-gerais': 'mg',
    'bahia': 'ba',
    'espirito-santo': 'es',
    'parana': 'pr',
    'santa-catarina': 'sc',
    'goias': 'go',
    'pernambuco': 'pe',
    'ceara': 'ce',
    'mato-grosso': 'mt',
    'mato-grosso-do-sul': 'ms',
    'pariba': 'pb',
    'sergipe': 'se',
    'alagoas': 'al',
    'amazonas': 'am',
    'acre': 'ac',
    'rondonia': 'ro',
    'maranhao': 'ma',
    'piaui': 'pi',
    'rio-grande-do-sul': 'rs',
    'rio-grande-do-norte': 'rn',
    'tocantins': 'to',
    'distrito-federal': 'df'
}


df_itens_juncao['classificacao'] = df_itens_juncao['url'].str.extract(r'(?<=\.com\/)([^\/]+)')
df_itens_juncao['classificacao'] = df_itens_juncao['classificacao'].replace(abreviacoes_estados)

In [ ]:
estados_br = [
    "ac", "al", "ap", "am", "ba", "ce", "df", "es", "go", "ma",
    "mt", "ms", "mg", "pa", "pb", "pr", "pe", "pi", "rj", "rn",
    "rs", "ro", "rr", "sc", "sp", "se", "to"
]

jornais_programas = [
    'jornal-nacional', 'jornal-hoje', 'bom-dia-brasil', 'jornal-da-globo',
    'hora1', 'fantastico', 'profissao-reporter', 'globo-reporter',
    'fato-ou-fake', 'resumo-do-dia', 'agenda-do-dia','globonews',
    'g1-explica', 'g1-15-anos', 'vc-no-g1', 'app-g1', 'ultimas-noticias'
]


temas = [
    'politica', 'economia', 'meio-ambiente', 'saude', 'ciencia', 'tecnologia',
    'empreendedorismo', 'trabalho-e-carreira', 'educacao', 'ciencia-e-saude',
    'natureza', 'turismo-e-viagem', 'mobilidade', 'loterias', 'loteria',
    'concursos-e-emprego', 'previsao-do-tempo', 'inovacao', 'tecnologia', 'como-sera'
]

eventos = [
    'carnaval', 'dia-das-mulheres', 'consciencia-negra', 'rock-in-rio'
]

curiosidades = [
    'adnet-na-cpi', 'que-meme-e-esse', 'e-ou-nao-e'
]

especiais_blogs = [
    'especial-publicitario', 'especiais', 'blogs-e-colunas', 'monitor-da-violencia',
    'retrospectiva', 'retrospectiva-g1', 'e-ou-nao-e', 'segue-o-fio', 'expedicaorio',
    'fique-em-casa', 'agora-e-assim'
]

entretenimento = [
    'musica', 'pop-arte', 'comida-di-buteco', 'que-meme-e-esse', 'planeta-bizarro',
    'olha-que-legal'
]

def categorizar(valor):
    if valor in estados_br:
        return 'Estados'
    elif valor in jornais_programas:
        return 'Jornal/Programa'
    elif valor in temas:
        return 'Temas'
    elif valor in eventos:
        return 'Eventos'
    elif valor in curiosidades:
        return 'Curiosidades'
    elif valor in especiais_blogs:
        return 'Especiais Blogs'
    elif valor in entretenimento:
        return 'Entreterimento'
    else:
        return 'Outro'

# Criando uma nova coluna com a categoria de cada valor
df_itens_juncao['agrupamento'] = df_itens_juncao['classificacao'].apply(categorizar)

In [ ]:
df_itens_juncao.head()

,page,url,issued,modified,title,body,caption,classificacao,agrupamento
0,7371a9b5-5824-4c57-8704-00a74feebe79,http://g1.globo.com/al/alagoas/noticia/2018/09...,2018-09-13 14:52:55+00:00,2018-09-14 16:14:49+00:00,Corpo de motorista da Uber é encontrado em can...,Corpo de motorista de aplicativo desaparecido ...,"Segundo a polícia, Antônio Vitor foi solicitad...",al,Estados
1,7a5ea08f-4583-49e2-ba52-a71999443f7b,http://g1.globo.com/am/amazonas/noticia/detent...,2018-05-20 20:42:40+00:00,2018-05-20 20:42:40+00:00,Detento recapturado após fuga por túnel volta ...,Detento disse que passou nome falso ao dar ent...,Ele tinha registro em presídio com nome falso....,am,Estados
2,6afc8bbb-4f36-43d5-8a44-a2917df5621a,http://g1.globo.com/ap/amapa/noticia/audios-mo...,2017-07-30 00:37:17+00:00,2017-07-30 00:48:42+00:00,Áudios mostram conversa entre bandidos durante...,Áudios mostram possível conversa entre bandido...,Revista realizada na sexta-feira (28) no Iapen...,ap,Estados
3,5cc3bd27-80c7-457d-a807-2e8e7fddf031,http://g1.globo.com/ap/amapa/noticia/2020/11/0...,2020-11-06 12:54:00+00:00,2020-11-12 21:22:52+00:00,FOTOS: Apagão no Amapá,"Moradores da capital do Amapá, em Macapá, faze...",Incêndio em subestação de energia deixa 13 dos...,ap,Estados
4,d6956177-db96-42f5-9f68-dd0d6e930661,http://g1.globo.com/ap/amapa/noticia/2019/05/2...,2019-05-27 13:43:03+00:00,2019-05-27 18:19:06+00:00,Profissionais da educação no AP paralisam ativ...,Profissionais da educação paralisam atividades...,Ato comprometeu aulas em escolas nesta segunda...,ap,Estados


In [ ]:
""""
----------------------- AVISO ------------------------

Esta celula demora cerca de 40 min para ser executada
e consome boa parte da RAM disponivel. Mude o diretorio
de salvar o arquivo caso não utilize o Google Drive.

------------------------------------------------------
"""

stopwords = nltk.corpus.stopwords.words("portuguese")

# Inicialize o vetor de TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords)

# Ajuste o modelo TF-IDF no conteúdo das notícias
tfidf_matrix = tfidf_vectorizer.fit_transform(df_itens_juncao['body'])

# Pegue as palavras que foram extraídas
words = tfidf_vectorizer.get_feature_names_out()

# Crie um DataFrame com as palavras mais relevantes por notícia
top_words = []
for i in range(tfidf_matrix.shape[0]):
    # Obtenha as pontuações do TF-IDF para cada palavra da notícia
    row = tfidf_matrix[i].toarray().flatten()
    # Pegue os índices das 5 palavras com maior valor de TF-IDF
    top_indices = row.argsort()[-5:][::-1]
    top_words.append([words[idx] for idx in top_indices])

# Adicione as 5 palavras mais importantes para cada notícia ao DataFrame
df_itens_juncao['top_5_palavras'] = top_words
df_itens_juncao.to_parquet(f"{dir_arq_resultado_c}/itens_finalv2.parquet", index=False)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-36-64a595b1b4d7>", line 9, in <cell line: 0>
    tfidf_matrix = tfidf_vectorizer.fit_transform(df_itens_juncao['body'])
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py", line 2104, in fit_transform
    X = super().fit_transform(raw_documents)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py", line 1376, in fit_transform
    vocabulary, X = self._count_vocab(raw_documents, self.fixed_vocabulary_)
      

TypeError: object of type 'NoneType' has no len()

## Parte 2 - Utilização dataframe | df_itens_juncao

In [ ]:
# ----- Caso ja tenha executado a celula acima, execute desta celula para baixo -----
df_itens_juncao = pd.read_parquet('/content/drive/My Drive/datathon/resultados_csv/itens_finalv2.parquet')

In [ ]:
df_itens_juncao.head()

,page,url,issued,modified,title,body,caption,classificacao,agrupamento,top_5_palavras
0,7371a9b5-5824-4c57-8704-00a74feebe79,http://g1.globo.com/al/alagoas/noticia/2018/09...,2018-09-13 14:52:55+00:00,2018-09-14 16:14:49+00:00,Corpo de motorista da Uber é encontrado em can...,Corpo de motorista de aplicativo desaparecido ...,"Segundo a polícia, Antônio Vitor foi solicitad...",al,Estados,"[encontrado, roteiro, alagoas, canavial, deodoro]"
1,7a5ea08f-4583-49e2-ba52-a71999443f7b,http://g1.globo.com/am/amazonas/noticia/detent...,2018-05-20 20:42:40+00:00,2018-05-20 20:42:40+00:00,Detento recapturado após fuga por túnel volta ...,Detento disse que passou nome falso ao dar ent...,Ele tinha registro em presídio com nome falso....,am,Estados,"[fuga, maruan, seap, foragidos, cdpm]"
2,6afc8bbb-4f36-43d5-8a44-a2917df5621a,http://g1.globo.com/ap/amapa/noticia/audios-mo...,2017-07-30 00:37:17+00:00,2017-07-30 00:48:42+00:00,Áudios mostram conversa entre bandidos durante...,Áudios mostram possível conversa entre bandido...,Revista realizada na sexta-feira (28) no Iapen...,ap,Estados,"[g5, g4, iapen, fuga, conversa]"
3,5cc3bd27-80c7-457d-a807-2e8e7fddf031,http://g1.globo.com/ap/amapa/noticia/2020/11/0...,2020-11-06 12:54:00+00:00,2020-11-12 21:22:52+00:00,FOTOS: Apagão no Amapá,"Moradores da capital do Amapá, em Macapá, faze...",Incêndio em subestação de energia deixa 13 dos...,ap,Estados,"[macapá, amazônica, apagão, amapá, rede]"
4,d6956177-db96-42f5-9f68-dd0d6e930661,http://g1.globo.com/ap/amapa/noticia/2019/05/2...,2019-05-27 13:43:03+00:00,2019-05-27 18:19:06+00:00,Profissionais da educação no AP paralisam ativ...,Profissionais da educação paralisam atividades...,Ato comprometeu aulas em escolas nesta segunda...,ap,Estados,"[sinsepeap, paralisação, odiléa, gea, amapá]"


In [ ]:
df_itens_exp = df_itens_juncao.drop(columns=['url', 'modified', 'title', 'body', 'caption'])
df_itens_exp['issued'] = pd.to_datetime(df_itens_exp['issued']).dt.date

In [ ]:
df_itens_exp.head()

,page,issued,classificacao,agrupamento,top_5_palavras
0,7371a9b5-5824-4c57-8704-00a74feebe79,2018-09-13,al,Estados,"[encontrado, roteiro, alagoas, canavial, deodoro]"
1,7a5ea08f-4583-49e2-ba52-a71999443f7b,2018-05-20,am,Estados,"[fuga, maruan, seap, foragidos, cdpm]"
2,6afc8bbb-4f36-43d5-8a44-a2917df5621a,2017-07-30,ap,Estados,"[g5, g4, iapen, fuga, conversa]"
3,5cc3bd27-80c7-457d-a807-2e8e7fddf031,2020-11-06,ap,Estados,"[macapá, amazônica, apagão, amapá, rede]"
4,d6956177-db96-42f5-9f68-dd0d6e930661,2019-05-27,ap,Estados,"[sinsepeap, paralisação, odiléa, gea, amapá]"


# Arquivo | treino_parte.csv

In [ ]:
df_treino_juncao = pd.read_parquet("/content/drive/My Drive/datathon/resultados_csv/treino_final.parquet")

In [ ]:
df_validacao = pd.read_csv("/content/drive/My Drive/datathon/resultados_csv/validacao.csv")

In [ ]:
df_treino_juncao.head()

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,"c8aab885-433d-4e46-8066-479f40ba7fb2, 68d2039c...","1657146417045, 1657146605778, 1657146698738","76, 38, 41","20380, 21184, 35438","50.3, 18.18, 16.46","2, 1, 1","1657146417045, 1657146605778, 1657146698738"
1,2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...,Non-Logged,60,"3325b5a1-979a-4cb3-82b6-63905c9edbe8, fe856057...","1656684240278, 1656761266729, 1656761528085, 1...","7, 80, 2, 1, 7, 62, 26, 44, 4, 4, 14, 45, 13, ...","6049, 210489, 8672, 10000, 30000, 123007, 9965...","25.35, 45.66, 35.3, 28.05, 36.53, 47.57, 55.33...","1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1...","1656684240278, 1656761266729, 1656761528085, 1..."
2,0adffd7450d3b9840d8c6215f0569ad942e782fb19b805...,Logged,107,"04756569-593e-4133-a95a-83d35d43dbbd, 29b6b142...","1656678946256, 1656701076495, 1656701882565, 1...","0, 0, 0, 0, 0, 44, 0, 0, 2, 1, 0, 0, 0, 44, 0,...","311274, 140000, 32515, 157018, 118689, 159243,...","67.58, 47.22, 41.52, 63.09, 51.38, 65.11, 71.9...","1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1...","1656678946256, 1656701076495, 1656701882565, 1..."
3,c1e8d644329a78ea1f994292db624c57980b2886cfbc2d...,Non-Logged,56,"1f2b9c2f-a2d2-4192-b009-09065da8ec23, 04756569...","1658333312180, 1658404553818, 1658408449062, 1...","8, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 1, 1...","182696, 91925, 30000, 273655, 126409, 42980, 1...","58.26, 72.66, 22.57, 59.89, 40.36, 36.35, 14.7...","1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1...","1658333312180, 1658404553818, 1658408449062, 1..."
4,e777d1f31d4d955b63d60acc13df336d3903f52ab8f8f4...,Non-Logged,4,"bebdeb3e-1699-43e0-a1b8-989f5a6ab679, f4b484a7...","1658766608801, 1658766608801, 1660084035094, 1...","579, 579, 7, 2","801396, 801396, 10000, 10000","78.74, 78.74, 16.71, 9.34","7, 7, 1, 1","1658766608801, 1658766608801, 1660084035094, 1..."


In [ ]:
df_validacao.head()

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8'\n '01c...,[1660533136590 1660672113513]
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,['77901133-aee7-4f7b-afc0-652231d76fe9'],[1660556860253]
2,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,['857aa90f-a7ec-410d-ba82-dfa4f85d4e71'],[1660561649242]
3,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,Logged,['b7b90e18-7613-4ca0-a8fc-fd69addfcd85'\n '835...,[1660533830245 1660540831707 1660542659111 166...
4,a120515626fe5d12b22b7d5a7c5008912cc69284aa26cc...,Logged,['9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6'\n 'b8e...,[1660548813953 1660572329731 1660594848200]


In [ ]:
def tratamento_treino(df_treino, df_vali, df_itens, user_type):
  df_treino = df_treino.drop(columns=['timestampHistory_new'], axis=1)
  df_treino = df_treino[df_treino['userType'] == user_type]
  df_vali = df_vali[df_vali['userType'] == user_type]
  df_treino2 = df_treino[df_treino['userId'].isin(df_vali['userId'])]

  cols_to_explode = ["history", "timestampHistory", "numberOfClicksHistory",
                    "timeOnPageHistory", "scrollPercentageHistory",
                    "pageVisitsCountHistory"]

  for col in cols_to_explode:
      df_treino2[col] =df_treino2[col].str.split(", ")

  # Aplicar explode em todas as colunas necessárias
  df_treino_3 = df_treino2.explode(cols_to_explode, ignore_index=True)

  df_treino_4 = df_treino_3.merge(df_itens, left_on='history', right_on='page', how='left')

  frequencias = df_treino_4['classificacao'].value_counts()
  valores_para_manter = frequencias[frequencias > 1].index
  df_treino_4 = df_treino_4[df_treino_4['classificacao'].isin(valores_para_manter)]

  df_treino_4 = df_treino_4.dropna(subset=['classificacao'])
  df_lixo, df_treino_5 = train_test_split(df_treino_4, test_size=0.02, stratify=df_treino_4['classificacao'], random_state=42)

  return df_treino_5

In [ ]:
df_treino_exp_logged = tratamento_treino(df_treino=df_treino_juncao, df_vali=df_validacao, df_itens=df_itens_exp, user_type="Logged")

<ipython-input-8-1004f0248b43>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_treino2[col] =df_treino2[col].str.split(", ")


In [ ]:
df_treino_exp_logged.head()

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,page,issued,classificacao,agrupamento,top_5_palavras
1819621,c1bcbb88d03942bb0422aa9ea0b315aa6af5779da9270e...,Logged,670,50f85bf5-c153-4611-bf35-2f781ae4e234,1660151400938,1,175782,41.06,1,50f85bf5-c153-4611-bf35-2f781ae4e234,2022-07-07,pr,Estados,"[manuela, intimação, grávida, samuel, intimada]"
1704058,a893820bc7f90a3c8ec4f6b364a35e4c00dfe203ea368d...,Logged,185,ecd63535-1875-4f84-b7c1-3aabcadd316f,1657967489711,22,54057,46.51,1,ecd63535-1875-4f84-b7c1-3aabcadd316f,2022-08-03,politica,Temas,"[teletrabalho, jornada, alimentação, trabalhad..."
25246,0d1c5530309aa7e590a08037f04a10f083b9f1548de1bf...,Logged,214,6867c681-1abc-43ae-97aa-7015ea14b226,1660335491062,4,222056,38.76,4,6867c681-1abc-43ae-97aa-7015ea14b226,2022-07-01,economia,Temas,"[dólar, pec, ipp, inflação, 3206]"
2225012,0072fd5984992cb08b4211fd8f7a850b4d4e50ae3dd0fe...,Logged,75,c21004f0-4d4b-4882-8718-e1d6d276b175,1660220508981,0,15997,28.15,1,c21004f0-4d4b-4882-8718-e1d6d276b175,2022-07-06,sp,Estados,"[gabi, fatura, brandt, 377, yanka]"
858475,e7acc173dcd9d65d3cdf847e4514c796f8aa77863cc7b9...,Logged,79,6311c9a0-e8eb-4cb4-b87e-b2f2afeb9e81,1656831371977,13,70662,52.78,1,6311c9a0-e8eb-4cb4-b87e-b2f2afeb9e81,2022-08-10,df,Estados,"[esbarrão, thales, colega, diogo, discussão]"


In [ ]:
df_treino_exp_nonlogged = tratamento_treino(df_treino=df_treino_juncao, df_vali=df_validacao, df_itens=df_itens_exp, user_type="Non-Logged")

<ipython-input-8-1004f0248b43>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_treino2[col] =df_treino2[col].str.split(", ")


In [ ]:
df_treino_exp_nonlogged.head()

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,page,issued,classificacao,agrupamento,top_5_palavras
1679177,605f9303a6fcc007cf20ddfb82197e9fb3c8a09dbf7f3c...,Non-Logged,685,2de36771-38b8-4e2c-a2b7-d00590dd9009,1658233978404,0,40000,74.92,1,2de36771-38b8-4e2c-a2b7-d00590dd9009,2022-07-26,sc,Estados,"[bebê, blumenau, encontrados, mãe, delegado]"
1801549,f06672a2ca2c8c0206c28c197749ac5a3b7f97094bca90...,Non-Logged,169,296094f4-2b02-4102-b2f9-570bfd93d99a,1659404684629,1,97092,53.87,1,296094f4-2b02-4102-b2f9-570bfd93d99a,2022-07-18,mg,Estados,"[salinas, mgc, 342, rhonann, ocupantes]"
1357930,303b779aa3dda0cf0be87a5d75fbaf3adc5618a69b98dc...,Non-Logged,137,b3aa822f-fc30-4198-b484-e5d29f94641d,1658336129305,0,40000,44.47,1,b3aa822f-fc30-4198-b484-e5d29f94641d,2022-07-22,mundo,Outro,"[padilla, jalisco, zapopan, méxico, twitter]"
2276945,b3df95c8f32a5f2ce827574ac7ffc423110dd5adfa344c...,Non-Logged,693,bf550400-b308-41e3-ba0d-1018ad1c7036,1657118621183,0,20696,39.64,1,bf550400-b308-41e3-ba0d-1018ad1c7036,2022-08-03,mundo,Outro,"[taiwan, china, pelosi, pequim, nancy]"
1827397,a81a58a06df9c8df9626bb5fd1f8fcb9c5dd38387618f7...,Non-Logged,38,29b6b142-4173-4ec4-832f-7d0a32255c10,1660056021757,0,25592,18.87,1,29b6b142-4173-4ec4-832f-7d0a32255c10,2022-08-10,rj,Estados,"[sabine, boghici, tarsila, quadros, poente]"


In [ ]:
df_treino_exp_logged.to_parquet(f"{dir_arq_resultado_c}/treino_exp_logged.parquet", index=False)

In [ ]:
df_treino_exp_nonlogged.to_parquet(f"{dir_arq_resultado_c}/treino_exp_nonlogged.parquet", index=False)

# Treinamento Modelo - Logados | SVD - Matriz de Interação

In [ ]:
df_treino_exp_logged = pd.read_parquet(f"{dir_arq_resultado_c}/treino_exp_logged.parquet")

In [ ]:
df_treino_exp_logged = df_treino_exp_logged.drop(columns=['userType', 'historySize', 'page', 'timestampHistory', 'pageVisitsCountHistory'])

In [ ]:
df_treino_exp_logged.head()

,userId,history,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,issued,classificacao,agrupamento,top_5_palavras
0,c1bcbb88d03942bb0422aa9ea0b315aa6af5779da9270e...,50f85bf5-c153-4611-bf35-2f781ae4e234,1,175782,41.06,2022-07-07,pr,Estados,"[manuela, intimação, grávida, samuel, intimada]"
1,a893820bc7f90a3c8ec4f6b364a35e4c00dfe203ea368d...,ecd63535-1875-4f84-b7c1-3aabcadd316f,22,54057,46.51,2022-08-03,politica,Temas,"[teletrabalho, jornada, alimentação, trabalhad..."
2,0d1c5530309aa7e590a08037f04a10f083b9f1548de1bf...,6867c681-1abc-43ae-97aa-7015ea14b226,4,222056,38.76,2022-07-01,economia,Temas,"[dólar, pec, ipp, inflação, 3206]"
3,0072fd5984992cb08b4211fd8f7a850b4d4e50ae3dd0fe...,c21004f0-4d4b-4882-8718-e1d6d276b175,0,15997,28.15,2022-07-06,sp,Estados,"[gabi, fatura, brandt, 377, yanka]"
4,e7acc173dcd9d65d3cdf847e4514c796f8aa77863cc7b9...,6311c9a0-e8eb-4cb4-b87e-b2f2afeb9e81,13,70662,52.78,2022-08-10,df,Estados,"[esbarrão, thales, colega, diogo, discussão]"


In [ ]:
# Filtrar notícias do último mês
df_treino_exp_logged['issued'] = pd.to_datetime(df_treino_exp_logged['issued'])
ultimo_mes = df_treino_exp_logged['issued'].max() - pd.DateOffset(days=30)
df_recente = df_treino_exp_logged[df_treino_exp_logged['issued'] >= ultimo_mes]

In [ ]:
# Selecionar as notícias mais populares
num_noticias_populares = 10  # Defina quantas notícias populares considerar
df_treino_exp_logged["numberOfClicksHistory"] = pd.to_numeric(df_treino_exp_logged["numberOfClicksHistory"], errors='coerce')
df_popular = df_treino_exp_logged.nlargest(num_noticias_populares, 'numberOfClicksHistory')

In [ ]:
# Criar o conjunto de notícias relevantes
df_relevante = pd.concat([df_recente, df_popular]).drop_duplicates(subset=[col for col in df_recente.columns if col != 'top_5_palavras'])

In [ ]:
# Criar matriz de interação
interaction_matrix = df_treino_exp_logged.pivot_table(index='userId', columns='history', values='numberOfClicksHistory', fill_value=0)

In [ ]:
# Aplicar SVD para fatoração
svd = TruncatedSVD(n_components=50)
user_factors = svd.fit_transform(interaction_matrix)
news_factors = svd.components_.T

# Normalizar os fatores para evitar vieses
scaler = StandardScaler()
user_factors = scaler.fit_transform(user_factors)
news_factors = scaler.fit_transform(news_factors)

In [ ]:
# Criar um DataFrame para os fatores das notícias
news_factors_df = pd.DataFrame(news_factors, index=interaction_matrix.columns)

In [ ]:
# Função para recomendar notícias com base na similaridade
def recomendar_noticias_para_usuario(user_id, top_n=5):
    if user_id not in interaction_matrix.index:
        return "Usuário não encontrado"

    # Obter a representação latente do usuário
    user_vector = user_factors[interaction_matrix.index.get_loc(user_id)].reshape(1, -1)

    # Calcular similaridade com todas as notícias
    similaridades = linear_kernel(user_vector, news_factors_df.values).flatten()

    # Criar um DataFrame de similaridade
    similaridade_series = pd.Series(similaridades, index=news_factors_df.index)

    # Filtrar para notícias recentes e populares
    noticias_validas = df_relevante['history'].values
    recomendacoes = similaridade_series[noticias_validas].nlargest(top_n)

    return recomendacoes.index.tolist()

In [ ]:
# Exemplo de recomendação para um usuário
usuario_exemplo = df_treino_exp_logged['userId'].iloc[0]
recomendacoes = recomendar_noticias_para_usuario(usuario_exemplo)
print(f"Notícias recomendadas para {usuario_exemplo}: {recomendacoes}")

Notícias recomendadas para c1bcbb88d03942bb0422aa9ea0b315aa6af5779da9270e212f0b91b4011940f5: ['6a83890a-d9e9-4f6b-a6c6-90d031785bbf', '6a83890a-d9e9-4f6b-a6c6-90d031785bbf', '6a83890a-d9e9-4f6b-a6c6-90d031785bbf', '6a83890a-d9e9-4f6b-a6c6-90d031785bbf', '6a83890a-d9e9-4f6b-a6c6-90d031785bbf']


# Treinamento Modelo - Não Logados | TF-IDF/Coseno

In [16]:
df_treino_exp_nonlogged = pd.read_parquet(f"{dir_arq_resultado_c}/treino_exp_nonlogged.parquet")

In [ ]:
df_treino_exp_nonlogged.head()

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,page,issued,classificacao,agrupamento,top_5_palavras
0,605f9303a6fcc007cf20ddfb82197e9fb3c8a09dbf7f3c...,Non-Logged,685,2de36771-38b8-4e2c-a2b7-d00590dd9009,1658233978404,0,40000,74.92,1,2de36771-38b8-4e2c-a2b7-d00590dd9009,2022-07-26,sc,Estados,"[bebê, blumenau, encontrados, mãe, delegado]"
1,f06672a2ca2c8c0206c28c197749ac5a3b7f97094bca90...,Non-Logged,169,296094f4-2b02-4102-b2f9-570bfd93d99a,1659404684629,1,97092,53.87,1,296094f4-2b02-4102-b2f9-570bfd93d99a,2022-07-18,mg,Estados,"[salinas, mgc, 342, rhonann, ocupantes]"
2,303b779aa3dda0cf0be87a5d75fbaf3adc5618a69b98dc...,Non-Logged,137,b3aa822f-fc30-4198-b484-e5d29f94641d,1658336129305,0,40000,44.47,1,b3aa822f-fc30-4198-b484-e5d29f94641d,2022-07-22,mundo,Outro,"[padilla, jalisco, zapopan, méxico, twitter]"
3,b3df95c8f32a5f2ce827574ac7ffc423110dd5adfa344c...,Non-Logged,693,bf550400-b308-41e3-ba0d-1018ad1c7036,1657118621183,0,20696,39.64,1,bf550400-b308-41e3-ba0d-1018ad1c7036,2022-08-03,mundo,Outro,"[taiwan, china, pelosi, pequim, nancy]"
4,a81a58a06df9c8df9626bb5fd1f8fcb9c5dd38387618f7...,Non-Logged,38,29b6b142-4173-4ec4-832f-7d0a32255c10,1660056021757,0,25592,18.87,1,29b6b142-4173-4ec4-832f-7d0a32255c10,2022-08-10,rj,Estados,"[sabine, boghici, tarsila, quadros, poente]"


In [ ]:
df_treino_exp_nonlogged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47543 entries, 0 to 47542
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   userId                   47543 non-null  object
 1   userType                 47543 non-null  object
 2   historySize              47543 non-null  int64 
 3   history                  47543 non-null  object
 4   timestampHistory         47543 non-null  object
 5   numberOfClicksHistory    47543 non-null  object
 6   timeOnPageHistory        47543 non-null  object
 7   scrollPercentageHistory  47543 non-null  object
 8   pageVisitsCountHistory   47543 non-null  object
 9   page                     47543 non-null  object
 10  issued                   47543 non-null  object
 11  classificacao            47543 non-null  object
 12  agrupamento              47543 non-null  object
 13  top_5_palavras           47543 non-null  object
dtypes: int64(1), object(13)
memory usage: 

In [ ]:
df_treino_exp_nonlogged = df_treino_exp_nonlogged.drop(columns=['userType', 'historySize', 'timestampHistory', 'timeOnPageHistory', 'scrollPercentageHistory', 'page'])

In [ ]:
# Filtrar notícias do último mês
df_treino_exp_nonlogged['issued'] = pd.to_datetime(df_treino_exp_nonlogged['issued'])
ultimo_mes = df_treino_exp_nonlogged['issued'].max() - pd.DateOffset(days=30)
df_recente = df_treino_exp_nonlogged[df_treino_exp_nonlogged['issued'] >= ultimo_mes]

In [ ]:
# Selecionar as notícias mais populares
num_noticias_populares = 100  # Defina quantas notícias populares considerar
df_treino_exp_nonlogged["numberOfClicksHistory"] = pd.to_numeric(df_treino_exp_nonlogged["numberOfClicksHistory"], errors='coerce')
df_popular = df_treino_exp_nonlogged.nlargest(num_noticias_populares, 'numberOfClicksHistory')

In [ ]:
# Criar o conjunto de notícias relevantes
df_relevante = pd.concat([df_recente, df_popular]).drop_duplicates(subset=[col for col in df_recente.columns if col != 'top_5_palavras'])

In [ ]:
# Combinar as colunas de texto para TF-IDF
df_relevante['conteudo'] = df_relevante['top_5_palavras'] + " " + df_relevante['classificacao'] + " " + df_relevante['agrupamento']

In [ ]:
# Vetorização com TF-IDF
df_relevante['conteudo'] = df_relevante['conteudo'].astype(str)
df_relevante = df_relevante.drop(columns=['top_5_palavras', 'numberOfClicksHistory', 'pageVisitsCountHistory', 'issued'])
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_relevante['conteudo'])

In [ ]:
svd = TruncatedSVD(n_components=100)  # Reduzindo para 100 dimensões
reduced_matrix = svd.fit_transform(tfidf_matrix)

In [ ]:
similaridade = linear_kernel(reduced_matrix, reduced_matrix)

In [17]:
# Criar um DataFrame de similaridade
indices_noticias = df_relevante['history'].values
similaridade_df = pd.DataFrame(similaridade, index=indices_noticias, columns=indices_noticias)

In [ ]:
# Função para recomendar notícias similares
def recomendar_noticias2(df, usuario_id, top_n=5):
    df = df[df['userId'] == usuario_id]
    lista_unicos = df['history'].unique().tolist()
    lista_noticias = []
    for noticia in lista_unicos:
      if noticia in similaridade_df.index.unique().to_list():
        similares = similaridade_df[noticia].squeeze().iloc[1:5+1]
        lista_noticias.append(similares.index.to_list())
      else:
        lista_noticias.append(None)
    return lista_noticias

In [ ]:
df_treino_exp_nonlogged['userId'][0]

'605f9303a6fcc007cf20ddfb82197e9fb3c8a09dbf7f3c90b8ae9105fa157a6d'

In [ ]:
recomendacoes = recomendar_noticias2(df=df_treino_exp_nonlogged, usuario_id="605f9303a6fcc007cf20ddfb82197e9fb3c8a09dbf7f3c90b8ae9105fa157a6d")
print(f"Notícias recomendadas o usuario: {recomendacoes}")

Notícias recomendadas o usuario: [['296094f4-2b02-4102-b2f9-570bfd93d99a', 'b3aa822f-fc30-4198-b484-e5d29f94641d', 'bf550400-b308-41e3-ba0d-1018ad1c7036', '29b6b142-4173-4ec4-832f-7d0a32255c10', 'aeab0e46-f1e4-41e9-821b-571255c41f69'], None, ['296094f4-2b02-4102-b2f9-570bfd93d99a', 'b3aa822f-fc30-4198-b484-e5d29f94641d', 'bf550400-b308-41e3-ba0d-1018ad1c7036', '29b6b142-4173-4ec4-832f-7d0a32255c10', 'aeab0e46-f1e4-41e9-821b-571255c41f69'], None, ['296094f4-2b02-4102-b2f9-570bfd93d99a', 'b3aa822f-fc30-4198-b484-e5d29f94641d', 'bf550400-b308-41e3-ba0d-1018ad1c7036', '29b6b142-4173-4ec4-832f-7d0a32255c10', 'aeab0e46-f1e4-41e9-821b-571255c41f69'], None, ['296094f4-2b02-4102-b2f9-570bfd93d99a', 'b3aa822f-fc30-4198-b484-e5d29f94641d', 'bf550400-b308-41e3-ba0d-1018ad1c7036', '29b6b142-4173-4ec4-832f-7d0a32255c10', 'aeab0e46-f1e4-41e9-821b-571255c41f69'], ['296094f4-2b02-4102-b2f9-570bfd93d99a', 'b3aa822f-fc30-4198-b484-e5d29f94641d', 'bf550400-b308-41e3-ba0d-1018ad1c7036', '29b6b142-4173-4ec4

# Testes

In [ ]:
df_treino_exp_nonlogged = pd.read_parquet(f"{dir_arq_resultado_c}/treino_exp_nonlogged.parquet")

In [ ]:
# Filtrar notícias do último mês
df_treino_exp_nonlogged['issued'] = pd.to_datetime(df_treino_exp_nonlogged['issued'])
ultimo_mes = df_treino_exp_nonlogged['issued'].max() - pd.DateOffset(days=30)
df_recente = df_treino_exp_nonlogged[df_treino_exp_nonlogged['issued'] >= ultimo_mes]

In [ ]:
# Selecionar as notícias mais populares
num_noticias_populares = 100  # Defina quantas notícias populares considerar
df_treino_exp_nonlogged["numberOfClicksHistory"] = pd.to_numeric(df_treino_exp_nonlogged["numberOfClicksHistory"], errors='coerce')
df_popular = df_treino_exp_nonlogged.nlargest(num_noticias_populares, 'numberOfClicksHistory')

In [ ]:
# Criar o conjunto de notícias relevantes
df_relevante = pd.concat([df_recente, df_popular]).drop_duplicates(subset=[col for col in df_recente.columns if col != 'top_5_palavras'])

In [ ]:
# Combinar as colunas de texto para TF-IDF
df_relevante['conteudo'] = df_relevante['top_5_palavras'] + " " + df_relevante['classificacao'] + " " + df_relevante['agrupamento']

In [ ]:
# Vetorização com TF-IDF
df_relevante['conteudo'] = df_relevante['conteudo'].astype(str)
df_relevante = df_relevante.drop(columns=['top_5_palavras', 'numberOfClicksHistory', 'pageVisitsCountHistory', 'issued'])
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_relevante['conteudo'])

In [ ]:
tfidf_matrix = csr_matrix(tfidf_matrix)

In [ ]:
# Aplicar SVD para fatores latentes das notícias
svd = TruncatedSVD(n_components=100)  # Mesma dimensionalidade do código original
item_factors = svd.fit_transform(tfidf_matrix)  # Fatores latentes das notícias

In [ ]:
# Mapear índices para IDs de notícias
indices_noticias = df_relevante['history'].values

In [ ]:
def recomendar_noticias_non_logged_svd(noticia_id, svd, item_factors, indices_noticias, top_n=5):
    """Recomenda notícias similares usando fatores latentes do SVD."""
    if noticia_id not in indices_noticias:
        return "Notícia não encontrada"

    # Encontrar o índice da notícia base
    noticia_index = np.where(indices_noticias == noticia_id)[0][0]

    # Vetor latente da notícia base
    noticia_vector = item_factors[noticia_index].reshape(1, -1)

    # Calcular escores de similaridade (produto interno) com todas as notícias
    scores = item_factors.dot(noticia_vector.T).flatten()

    # Ordenar e pegar os top N mais similares (excluindo a própria notícia)
    top_indices = np.argsort(scores)[::-1]
    top_indices = [i for i in top_indices if i != noticia_index][:top_n]
    recommended_news = indices_noticias[top_indices].tolist()

    return recommended_news

In [ ]:
noticia_exemplo = indices_noticias[0]  # Primeira notícia como exemplo
recomendacoes = recomendar_noticias_non_logged_svd(noticia_exemplo, svd, item_factors, indices_noticias)
print(f"Recomendações para notícia {noticia_exemplo}: {recomendacoes}")

Recomendações para notícia 2de36771-38b8-4e2c-a2b7-d00590dd9009: ['2e735dd9-49fc-472f-b724-aa236936546f', '2e735dd9-49fc-472f-b724-aa236936546f', '2e735dd9-49fc-472f-b724-aa236936546f', '2e735dd9-49fc-472f-b724-aa236936546f', '2e735dd9-49fc-472f-b724-aa236936546f']


# Codigo de geração dos arquivos

In [ ]:
# --- Celula de criação do dataset com todas as partes ---

# Listar todos os arquivos CSV na pasta
arquivos_csv = glob.glob(os.path.join(dir_arq_treino, "*.csv"))

# Criar um DataFrame vazio para armazenar os dados combinados
df_completo = pd.DataFrame()

# Percorrer os arquivos e adicioná-los ao DataFrame
for arquivo in arquivos_csv:
    df_temp = pd.read_csv(arquivo)  # Ler o CSV atual
    df_completo = pd.concat([df_completo, df_temp], ignore_index=True)  # Concatenar os dados

# Salvar o DataFrame final em um novo arquivo CSV
df_completo.to_parquet(f"{dir_arq_resultado_c}/treino_final.parquet", index=False)

print(f"Arquivo salvo em:{dir_arq_resultado_c}")

In [ ]:
# --- Celula de criação do dataset com a união de todos os arquivos de noticias pelo Google Drive (itens-parte.csv) ---

# Listar todos os arquivos CSV na pasta
arquivos_csv = glob.glob(os.path.join(dir_arq_itens, "*.csv"))

# Criar um DataFrame vazio para armazenar os dados combinados
df_noticias = pd.DataFrame()

# Percorrer os arquivos e adicioná-los ao DataFrame
for arquivo in arquivos_csv:
    df_temp = pd.read_csv(arquivo)  # Ler o CSV atual
    df_noticias = pd.concat([df_noticias, df_temp], ignore_index=True)  # Concatenar os dados

# Salvar o DataFrame final em um novo arquivo CSV
df_noticias.to_parquet(f"{dir_arq_resultado_c}/itens_final.parquet", index=False)

print(f"Arquivo salvo em:{dir_arq_resultado_c}")

# Codigos Antigos

## Codigos tratamento arquivos treino

In [ ]:
df_treino_juncao = df_treino_juncao.drop(columns=['timestampHistory_new'], axis=1)
df_treino_j_logged = df_treino_juncao[df_treino_juncao['userType'] == "Logged"]
df_validacao = df_validacao[df_validacao['userType'] == "Logged"]
df_treino_j2_logged = df_treino_j_logged[df_treino_j_logged['userId'].isin(df_validacao['userId'])]

In [ ]:
df_treino_j2_logged.head()

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
9,a64a6e34e27d6f312c0d15c7deba04d3c3466741e32936...,Logged,9,"f598b137-5a37-40d8-8245-1e0395b51e83, 782477a7...","1656901848993, 1656933782205, 1657545425043, 1...","1, 1, 44, 29, 1, 17, 1, 0, 0","10000, 61736, 53769, 50000, 60000, 50000, 3000...","6.81, 69.53, 27.82, 31.34, 17.35, 20.55, 26.63...","1, 1, 2, 1, 2, 2, 1, 1, 1"
12,1859975e492157269a52ca0981bebe6479613a5e378130...,Logged,90,"3c787cdc-99f7-4001-94ac-4edb2e971b94, aeab0e46...","1656688881587, 1656887307494, 1656952061733, 1...","10, 9, 0, 0, 0, 5, 0, 1, 1, 9, 0, 18, 5, 6, 18...","50000, 33323, 8033, 60611, 57376, 140000, 3228...","53.75, 50.79, 40.78, 60.74, 42.18, 64.21, 63.7...","1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1..."
14,3cba177af94a0f031fdcb34a629a063f14eec307abe8ae...,Logged,31,"55ab912a-2bac-46d9-9fcf-8e9be376f1b3, 8634c420...","1656676993569, 1656951090354, 1656952806162, 1...","12, 32, 30, 104, 138, 81, 111, 40, 8, 17, 38, ...","40000, 70261, 90227, 198472, 453157, 149911, 3...","100.0, 67.42, 82.29, 80.17, 96.72, 100.0, 96.7...","1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1..."
20,fa178fb1c49f3f68a98fb6629fab5f46f119e4a535e9e9...,Logged,82,"83ebda93-e1cf-422d-ae6b-18298c55ed8e, e69d567c...","1656943505228, 1656943560011, 1656944091878, 1...","4, 0, 1, 0, 0, 1, 0, 0, 0, 0, 9, 5, 0, 0, 0, 1...","110000, 9847, 68444, 70000, 90000, 19321, 3515...","52.19, 61.69, 53.05, 35.51, 33.25, 51.98, 51.5...","1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2..."
26,2ff2e481fc8b393aae2c294a9ed61b4df5e683f250c211...,Logged,16,"71e57a58-8bdf-42a3-9bcc-247bca49f356, ce591826...","1656976328027, 1657363902478, 1657363905375, 1...","1, 1, 2, 3, 1, 1, 26, 12, 17, 0, 2, 0, 2, 1, 0, 1","190366, 143294, 75529, 270000, 170315, 334192,...","70.12, 73.14, 30.03, 59.48, 71.7, 20.56, 23.3,...","1, 1, 1, 3, 1, 2, 3, 1, 3, 1, 2, 1, 1, 1, 1, 1"


In [ ]:
df_treino_j2_logged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46919 entries, 9 to 577922
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   userId                   46919 non-null  object
 1   userType                 46919 non-null  object
 2   historySize              46919 non-null  int64 
 3   history                  46919 non-null  object
 4   timestampHistory         46919 non-null  object
 5   numberOfClicksHistory    46919 non-null  object
 6   timeOnPageHistory        46919 non-null  object
 7   scrollPercentageHistory  46919 non-null  object
 8   pageVisitsCountHistory   46919 non-null  object
dtypes: int64(1), object(8)
memory usage: 3.6+ MB


In [ ]:
cols_to_explode = ["history", "timestampHistory", "numberOfClicksHistory",
                   "timeOnPageHistory", "scrollPercentageHistory",
                   "pageVisitsCountHistory"]

for col in cols_to_explode:
    df_treino_j2_logged[col] = df_treino_j2_logged[col].str.split(", ")

# Aplicar explode em todas as colunas necessárias
df_treino_exp_logged = df_treino_j2_logged.explode(cols_to_explode, ignore_index=True)

In [ ]:
df_treino_exp_logged.head()

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
0,a64a6e34e27d6f312c0d15c7deba04d3c3466741e32936...,Logged,9,f598b137-5a37-40d8-8245-1e0395b51e83,1656901848993,1,10000,6.81,1
1,a64a6e34e27d6f312c0d15c7deba04d3c3466741e32936...,Logged,9,782477a7-355a-4c01-9646-b82479638296,1656933782205,1,61736,69.53,1
2,a64a6e34e27d6f312c0d15c7deba04d3c3466741e32936...,Logged,9,30ceda2f-4fb2-4fa5-a244-f7f18988e1c9,1657545425043,44,53769,27.82,2
3,a64a6e34e27d6f312c0d15c7deba04d3c3466741e32936...,Logged,9,4e986ba0-ca6f-479f-979c-a111d0a2b7dc,1657660003487,29,50000,31.34,1
4,a64a6e34e27d6f312c0d15c7deba04d3c3466741e32936...,Logged,9,918903bc-b862-467f-90b3-9ede44ad0502,1658276322767,1,60000,17.35,2


In [ ]:
len(df_treino_exp_logged['history'].unique())

100527

In [ ]:
df_treino_exp2_logged = df_treino_exp_logged.merge(df_itens_exp, left_on='history', right_on='page', how='left')

In [ ]:
df_treino_exp2_logged.head()

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,page,issued,classificacao,agrupamento,top_5_palavras
0,a64a6e34e27d6f312c0d15c7deba04d3c3466741e32936...,Logged,9,f598b137-5a37-40d8-8245-1e0395b51e83,1656901848993,1,10000,6.81,1,f598b137-5a37-40d8-8245-1e0395b51e83,2022-08-02,ciencia,Temas,"[galáxia, webb, supertelescópio, james, nasa]"
1,a64a6e34e27d6f312c0d15c7deba04d3c3466741e32936...,Logged,9,782477a7-355a-4c01-9646-b82479638296,1656933782205,1,61736,69.53,1,782477a7-355a-4c01-9646-b82479638296,2021-12-16,politica,Temas,"[pesquisa, lula, anterior, ciro, sergio]"
2,a64a6e34e27d6f312c0d15c7deba04d3c3466741e32936...,Logged,9,30ceda2f-4fb2-4fa5-a244-f7f18988e1c9,1657545425043,44,53769,27.82,2,30ceda2f-4fb2-4fa5-a244-f7f18988e1c9,2021-07-10,jornal-nacional,Jornal/Programa,"[avaliavam, agora, maio, março, bolsonaro]"
3,a64a6e34e27d6f312c0d15c7deba04d3c3466741e32936...,Logged,9,4e986ba0-ca6f-479f-979c-a111d0a2b7dc,1657660003487,29,50000,31.34,1,4e986ba0-ca6f-479f-979c-a111d0a2b7dc,2022-07-21,rj,Estados,"[baleia, embarcação, barco, pescador, palu]"
4,a64a6e34e27d6f312c0d15c7deba04d3c3466741e32936...,Logged,9,918903bc-b862-467f-90b3-9ede44ad0502,1658276322767,1,60000,17.35,2,918903bc-b862-467f-90b3-9ede44ad0502,2018-10-08,rj,Estados,"[psl, dem, psol, deputados, mdb]"


In [ ]:
contagem_classificacao = df_treino_exp2_logged['classificacao'].value_counts()
print(contagem_classificacao.to_dict())

{'sp': 397719, 'rj': 389613, 'politica': 250696, 'mundo': 229284, 'economia': 128504, 'pop-arte': 92880, 'mg': 91934, 'pr': 90444, 'go': 75715, 'fantastico': 64987, 'sc': 60665, 'df': 60656, 'saude': 59085, 'ba': 52054, 'ms': 38273, 'pe': 36924, 'rs': 31401, 'tecnologia': 29052, 'ce': 27234, 'ciencia': 21695, 'jornal-nacional': 21629, 'pi': 17959, 'podcast': 15625, 'to': 13954, 'es': 13133, 'meio-ambiente': 13124, 'pa': 12888, 'am': 10379, 'mt': 9745, 'trabalho-e-carreira': 9519, 'educacao': 9494, 'pb': 9356, 'rn': 9228, 'turismo-e-viagem': 9140, 'ac': 8229, 'ma': 6961, 'al': 6832, 'ro': 6046, 'rr': 5857, 'ap': 5691, 'bemestar': 5387, 'fato-ou-fake': 5310, 'loterias': 4697, 'inovacao': 4692, 'se': 3342, 'olha-que-legal': 2858, 'hora1': 2369, 'bom-dia-brasil': 1853, 'jornal-hoje': 1510, 'jornal-da-globo': 1467, 'empreendedorismo': 1375, 'especiais': 1324, 'globonews': 1298, 'profissao-reporter': 1240, 'ciencia-e-saude': 964, 'natureza': 621, 'especial-publicitario': 552, 'globo-reporter

In [ ]:
"""
1 - Remover linha cujo classificação esta vazia, isso ocorre porque ha noticias que não estão na base de noticias
2 - Para o train_test_split rodar é necessario que as classificações tenham pelo menos 2 linhas, ultimas-noticias tem 1 linha
3 - Essa é a forma que foi encontrada de particionar o dataframe de forma que deixe os dados de forma equilibrada com classificacao
"""

df_treino_exp2_logged = df_treino_exp2_logged.dropna(subset=['classificacao'])
df_treino_exp2_logged = df_treino_exp2_logged[df_treino_exp2_logged['classificacao'] != 'ultimas-noticias']
df_lixo, df_treino_exp3_logged = train_test_split(df_treino_exp2_logged, test_size=0.02, stratify=df_treino_exp2_logged['classificacao'], random_state=42)

In [ ]:
# Verificar distribuições
print(df_treino_exp3_logged['classificacao'].value_counts(normalize=True))
print(df_lixo['classificacao'].value_counts(normalize=True))

classificacao
sp                 0.159937
rj                 0.156680
politica           0.100820
mundo              0.092214
economia           0.051677
                     ...   
blogs-e-colunas    0.000020
e-ou-nao-e         0.000020
como-sera          0.000020
resumo-do-dia      0.000020
g1-15-anos         0.000020
Name: proportion, Length: 73, dtype: float64
classificacao
sp                   1.599466e-01
rj                   1.566866e-01
politica             1.008197e-01
mundo                9.220857e-02
economia             5.167912e-02
                         ...     
correcoes            1.641467e-06
expedicaorio         1.231100e-06
brasil               8.207334e-07
previsao-do-tempo    8.207334e-07
paraiba              8.207334e-07
Name: proportion, Length: 87, dtype: float64


## Modelo antigo SVD

In [ ]:
# Nome dos dataframes "atualizados"
# Usuarios: df_treino_exp3
# Noticias: df_itens_exp

# Unindo com as notícias
df_treino_exp_logged = df_treino_exp_logged.drop(columns=['timestampHistory', 'pageVisitsCountHistory', 'page', 'classificacao', 'agrupamento', 'top_5_palavras', 'historySize', 'userType'])
df_total_logged = df_treino_exp_logged.copy()
df_total_logged["timeOnPageHistory"] = pd.to_numeric(df_total_logged["timeOnPageHistory"], errors='coerce')
df_total_logged["scrollPercentageHistory"] = pd.to_numeric(df_total_logged["scrollPercentageHistory"], errors='coerce')

In [ ]:
# Normalizando valores para criar o peso
max_time = df_total_logged["timeOnPageHistory"].max()
df_total_logged["normalized_time"] = df_total_logged["timeOnPageHistory"] / max_time
df_total_logged["normalized_scroll"] = df_total_logged["scrollPercentageHistory"] / 100

In [ ]:
# Fórmula do peso
alpha = 0.5  # Peso para o tempo na página
beta = 0.5   # Peso para o percentual de rolagem
df_total_logged["interaction_weight"] = (df_total_logged["normalized_time"] * alpha) + (df_total_logged["normalized_scroll"] * beta)

In [ ]:
df_total_logged.head()

,userId,history,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,issued,normalized_time,normalized_scroll,interaction_weight
0,c1bcbb88d03942bb0422aa9ea0b315aa6af5779da9270e...,50f85bf5-c153-4611-bf35-2f781ae4e234,1,175782,41.06,2022-07-07,0.029477,0.4106,0.220039
1,a893820bc7f90a3c8ec4f6b364a35e4c00dfe203ea368d...,ecd63535-1875-4f84-b7c1-3aabcadd316f,22,54057,46.51,2022-08-03,0.009065,0.4651,0.237082
2,0d1c5530309aa7e590a08037f04a10f083b9f1548de1bf...,6867c681-1abc-43ae-97aa-7015ea14b226,4,222056,38.76,2022-07-01,0.037237,0.3876,0.212419
3,0072fd5984992cb08b4211fd8f7a850b4d4e50ae3dd0fe...,c21004f0-4d4b-4882-8718-e1d6d276b175,0,15997,28.15,2022-07-06,0.002683,0.2815,0.142091
4,e7acc173dcd9d65d3cdf847e4514c796f8aa77863cc7b9...,6311c9a0-e8eb-4cb4-b87e-b2f2afeb9e81,13,70662,52.78,2022-08-10,0.011849,0.5278,0.269825


In [ ]:
#Ideia descartada
"""
news_counts = df_total2_logged['history'].value_counts()
popular_news = news_counts[news_counts >= 5].index  # Apenas notícias com pelo menos 5 interações
df_filtered = df_total2_logged[df_total2_logged['history'].isin(popular_news)]
"""

In [ ]:
# Criando a matriz de interação
interaction_matrix = df_total_logged.pivot(index="userId", columns="history", values="interaction_weight").fillna(0)

In [ ]:
# Convertendo para formato esparso
sparse_matrix = csr_matrix(interaction_matrix.values)

In [ ]:
# Aplicando SVD
svd = TruncatedSVD(n_components=20)  # Definir componentes latentes
scaler = StandardScaler()

user_factors = svd.fit_transform(sparse_matrix)
user_factors = scaler.fit_transform(user_factors)

# Exibir representações latentes dos usuários
print("Representações latentes dos usuários:")
print(user_factors)

Representações latentes dos usuários:
[[-0.00720785 -0.26059889 -0.32206494 ... -0.04794905 -0.04957116
   0.0267521 ]
 [-0.00720785 -0.20346005 -0.1879436  ... -0.01675097  0.18621717
   0.0325642 ]
 [-0.00720785 -0.06509184 -0.14305781 ...  0.03933271  0.14953528
   0.2595504 ]
 ...
 [-0.00720785 -0.24729963 -0.29453289 ... -0.02852513 -0.06048165
   0.0626452 ]
 [-0.00720785 -0.202934   -0.2275791  ...  0.05075067 -0.01405957
   0.00862335]
 [-0.00720785  0.14030841  0.05978735 ... -0.5572271  -3.56726569
   2.14524995]]


In [ ]:
len(user_factors)

19266

In [ ]:
# Função para recomendar notícias similares
def recomendar_noticias(noticia_id, top_n=5):
    if noticia_id not in similaridade_df.index:
        return "Notícia não encontrada"
    similares = similaridade_df[noticia_id].squeeze().iloc[1:top_n+1]
    return similares.index.tolist()

In [ ]:
# Exemplo de recomendação

noticia_exemplo = df_relevante['history'].iloc[0]
recomendacoes = recomendar_noticias(noticia_exemplo)
print(f"Notícias recomendadas para {noticia_exemplo}: {recomendacoes}")

Notícias recomendadas para 2de36771-38b8-4e2c-a2b7-d00590dd9009: ['296094f4-2b02-4102-b2f9-570bfd93d99a', 'b3aa822f-fc30-4198-b484-e5d29f94641d', 'bf550400-b308-41e3-ba0d-1018ad1c7036', '29b6b142-4173-4ec4-832f-7d0a32255c10', 'aeab0e46-f1e4-41e9-821b-571255c41f69']
